In [1]:
%pylab inline
import pandas as pd
from config import cnx # import the private db connection

Populating the interactive namespace from numpy and matplotlib


In [2]:
# hidden config file contains the database link
query = """SELECT * FROM salaries LIMIT 10"""
pd.read_sql_query(query, cnx)

,yearid,teamid,lgid,playerid,salary
0,1985,ATL,NL,barkele01,870000
1,1985,ATL,NL,bedrost01,550000
2,1985,ATL,NL,benedbr01,545000
3,1985,ATL,NL,campri01,633333
4,1985,ATL,NL,ceronri01,625000
5,1985,ATL,NL,chambch01,800000
6,1985,ATL,NL,dedmoje01,150000
7,1985,ATL,NL,forstte01,483333
8,1985,ATL,NL,garbege01,772000
9,1985,ATL,NL,harpete01,250000


In [3]:
# get all the tables into pandas dataframes
allstarfull = pd.read_sql_query("SELECT * FROM allstarfull", cnx)
fielding = pd.read_sql_query("SELECT * FROM fielding", cnx)
salaries = pd.read_sql_query("SELECT * FROM salaries", cnx)
schools = pd.read_sql_query("SELECT * FROM schools", cnx)
collegeplaying = pd.read_sql_query("SELECT * FROM collegeplaying", cnx)
teams = pd.read_sql_query("SELECT * FROM teams", cnx)

In [4]:
# a helper function to see if the sql query matches the pandas query
# prints the length of each result and returns the sum of matches for each column
def test_query(pd_q, sql_q):
    print 'pandas df length:', len(pd_q)
    print 'sql query length:', len(sql_q)
    return (pd_q == sql_q).sum()

In [5]:
""" 
Query Number 1.
Show all playerids and salaries with a salary in the year 1985 above 500k.
"""
q1_pd = salaries[(salaries.salary > 500000) & (salaries.yearid > 1985)][['playerid','salary']]

q1_query = "SELECT playerid, salary FROM salaries WHERE yearid > 1985 AND salary > 500000"
q1_sql = pd.read_sql_query(q1_query, cnx)

test_query(q1_sql.reset_index().sort(columns='playerid'),
           q1_pd.reset_index().sort(columns='playerid'))

pandas df length: 12627
sql query length: 12627


index           0
playerid    12627
salary      12627
dtype: int64

In [6]:
""" 
Query Number 2.
Show the team for each year that had a rank of 1.
"""
q2_pd = teams[teams['rank'] == 1][['teamid', 'yearid']]

q2_query = "SELECT teamid, yearid FROM teams WHERE rank = 1"
q2_sql = pd.read_sql_query(q2_query, cnx)

test_query(q2_sql.reset_index().sort(columns=['teamid', 'yearid']), 
           q2_pd.reset_index().sort(columns=['teamid', 'yearid']))

pandas df length: 406
sql query length: 406


index       0
teamid    406
yearid    406
dtype: int64

In [7]:
""" 
Query Number 3.
How many schools are in schoolstate of CT?
"""
q3_pd = len(schools[schools.schoolstate == 'CT'])

q3_query = "SELECT COUNT(schoolid) FROM schools WHERE schoolstate = 'CT'"
q3_sql = pd.read_sql_query(q3_query, cnx)

q3_sql == q3_pd

,count
0,True


In [8]:
""" 
Query Number 4.
How many schools are there in each state?
"""
q4_pd = schools[['schoolstate', 'schoolid']].groupby('schoolstate').count()

q4_query = "SELECT schoolstate, COUNT(schoolid) FROM schools GROUP BY schoolstate"
q4_sql = pd.read_sql_query(q4_query, cnx)
q4_sql = q4_sql.set_index('schoolstate').sort()
q4_sql.columns = ['schoolid']

test_query(q4_sql, q4_pd)

pandas df length: 49
sql query length: 49


schoolid    49
dtype: int64

In [9]:
""" 
Query Number 5.
What was the total spend on salaries by each team, each year?
"""
q5_pd = salaries[['teamid', 'yearid', 'salary']].groupby(['teamid', 'yearid']).sum()

q5_query = "SELECT teamid, yearid, SUM(salary) FROM salaries GROUP BY teamid, yearid"
q5_sql = pd.read_sql_query(q5_query, cnx)
q5_sql = q5_sql.groupby(['teamid', 'yearid']).sum().sort()
q5_sql.columns = ['salary']

test_query(q5_sql, q5_pd)

pandas df length: 860
sql query length: 860


salary    860
dtype: int64

In [10]:
""" 
Query Number 6.
Find all of the salaries of shortstops (fieldings, pos) for the year 2012.
"""
q6_pd = salaries[salaries.yearid == 2012][['playerid', 'salary']].merge(
                fielding[(fielding.pos == 'SS') & (fielding.yearid == 2012)][['playerid']],
                on=['playerid'],
                how="inner")

q6_query = """SELECT
    salaries.playerid, salaries.salary
 FROM salaries
 INNER JOIN fielding on salaries.playerid = fielding.playerid
 WHERE
    fielding.pos = 'SS' AND
    salaries.yearid = 2012 AND
    fielding.yearid = 2012"""
q6_sql = pd.read_sql_query(q6_query, cnx)

test_query(q6_sql, q6_pd)

pandas df length: 90
sql query length: 90


playerid    90
salary      90
dtype: int64

In [11]:
""" 
Query Number 7.
What is the first and last year played for each player?
"""
def min_max_year(df):
    min_year = df.yearid.min()
    max_year = df.yearid.max()
    return pd.Series([min_year, max_year], index = ['min', 'max'])
q7_pd = fielding.groupby('playerid').apply(min_max_year)

q7_query = 'SELECT playerid, MIN(yearid), MAX(yearid) FROM fielding GROUP BY playerid'
q7_sql = pd.read_sql_query(q7_query, cnx)

test_query(q7_sql.groupby('playerid').sum().sort(), q7_pd)

pandas df length: 18214
sql query length: 18214


min    18214
max    18214
dtype: int64

In [12]:
""" 
Query Number 8.
Who has played the most all star games?
"""
q8_pd = allstarfull[['playerid','gameid']].groupby('playerid').count()
q8_pd = q8_pd.sort('gameid', ascending=False).head(1)

q8_query = """
SELECT
    playerid,
    COUNT(gameid)
FROM allstarfull
GROUP BY playerid
ORDER BY  COUNT(gameid) DESC
LIMIT 1"""
q8_sql = pd.read_sql_query(q8_query, cnx)
q8_sql

q8_pd.index[0] == q8_sql.playerid.ix[0]

True

In [13]:
""" 
Query Number 9.
Which school has generated the most distinct players?

I interpret this as the number of players in college 
that also were in fielding and the college that had the most of these.

The problem with this is that there are some players who went to multiple schools
Ex. aardsda01 went to both Penn State and Rice
These people will be counted as having belonged to both schools
"""
q9_pd = schools[['schoolid', 'schoolname']].merge(collegeplaying, 
                                                  how='inner', 
                                                  left_on='schoolid', 
                                                  right_on='schoolid',)
q9_pd = q9_pd.merge(fielding, how='inner', left_on='playerid', right_on='playerid')
q9_pd = q9_pd[['schoolname', 'playerid']].drop_duplicates()
q9_pd = q9_pd.groupby('schoolname').count().sort('playerid', ascending=False)

q9_query = """
SELECT
    table2.schoolname,
    COUNT (table2.playerid)
FROM (SELECT DISTINCT
        schools.schoolid,
        schools.schoolname,
        fielding.playerid
    FROM collegeplaying
    INNER JOIN schools on collegeplaying.schoolID = schools.schoolid
    INNER JOIN fielding on collegeplaying.playerid = fielding.playerid) table2
GROUP BY table2.schoolname
ORDER BY COUNT(table2.playerid) DESC"""
q9_sql = pd.read_sql_query(q9_query, cnx)
q9_sql

q9_pd.index[0] == q9_sql.schoolname.ix[0]

True

In [32]:
"""
Query Number 10.
Which school has generated the most expensive players? (expensive defined by their first year's salary).

Side note:
 The top first year salary 'carpech02' is correct given the data provided, but the data is incorrect
 'carpech02' actually debuted in 2012 with a $482,000 salary (http://www.baseball-reference.com/players/c/carpech02.shtml_
 'carpech01' had teh $14M salaryin 2011 (http://www.baseball-reference.com/players/c/carpech01.shtml?redir)
"""

# get the first year for each player
first_year = salaries[['playerid', 'yearid']].groupby('playerid').min().reset_index()
# get the remaining salary table data for a player's first year
first_year = first_year.merge(salaries, how='left', on=['playerid', 'yearid'])
# merge with collegeplaying, but only the unique player/school tuples
college_to_pro = first_year.merge(collegeplaying[['playerid', 'schoolid']].drop_duplicates(),
                                  how = 'inner',
                                  on = 'playerid')
# merge in the schoolname and sort by salary
q10_pd = college_to_pro.merge(schools, how='inner', on='schoolid').sort('salary', ascending=False)

q10_query = """
SELECT
    first_year.playerid,
    first_year.min_year,
    salaries.salary,
    college_players.schoolname
FROM (SELECT
        playerid,
        MIN(yearid) AS min_year
     FROM salaries
     GROUP BY playerid) AS first_year
INNER JOIN salaries ON first_year.playerid = salaries.playerid 
                    AND first_year.min_year = salaries.yearid
INNER JOIN (SELECT DISTINCT
        playerid,
        collegeplaying.schoolid,
        schools.schoolname
    FROM collegeplaying 
    INNER JOIN schools ON schools.schoolid = collegeplaying.schoolid) as college_players
        ON college_players.playerid = first_year.playerid
ORDER BY salaries.salary DESC
"""
q10_sql = pd.read_sql_query(q10_query, cnx)

q10_sql['schoolname'].head(1) == q10_pd['schoolname'].head(1)

0    True
Name: schoolname, dtype: bool

In [40]:
"""
Query Number 11.
Show the 5 most expensive salaries for each team in the year 2014.
"""

q11_query = """
SELECT
    table2.teamid,
    table2.yearid,
    table2.playerid,
    table2.salary,
    table2.salary_rank
FROM 
    (SELECT
        teamid,
        yearid,
        playerid,
        salary,
        rank() OVER (PARTITION BY teamid ORDER BY salary DESC) AS "salary_rank"
    FROM salaries
    WHERE yearid = 2014) AS table2
WHERE table2.salary_rank < 6
"""
q11_sql = pd.read_sql_query(q11_query, cnx)
q11_sql

,teamid,yearid,playerid,salary,salary_rank
0,ARI,2014,pradoma01,11000000,1
1,ARI,2014,hillaa01,11000000,1
2,ARI,2014,montemi01,10000000,3
3,ARI,2014,arroybr01,9500000,4
4,ARI,2014,rossco01,9500000,4
5,ATL,2014,uptonju01,14250000,1
6,ATL,2014,santaer01,14100000,2
7,ATL,2014,uptonbj01,13450000,3
8,ATL,2014,ugglada01,13000000,4
9,ATL,2014,kimbrcr01,7000000,5


In [44]:
"""
Query Number 12.
Partition the average salaries by team and year, against year. 
Find players that were paid more than 1 standard deviation above
the average salary for that team and year. Show a count by playerid.
"""

q12_query = """
SELECT
    teamid,
    yearid,
    playerid,
    salary,
    avg(salary) OVER (PARTITION BY teamid, yearid) AS "avg_team_salary_yr"
FROM salaries
"""
q12_sql = pd.read_sql_query(q12_query, cnx)


TypeError: unhashable type

In [48]:
print q12_sql.loc[30:60, :]

   teamid  yearid   playerid   salary  avg_team_salary_yr
30    ANA    1997  watsoal01  1300000      1004370.064516
31    ANA    1998  percitr01  1125000      1214147.058824
32    ANA    1998  palmeor01   189000      1214147.058824
33    ANA    1998  bolicfr01   178000      1214147.058824
34    ANA    1998  anderga01  1500000      1214147.058824
35    ANA    1998  schmije01   170000      1214147.058824
36    ANA    1998  salmoti01  5000000      1214147.058824
37    ANA    1998  pritcch01   171000      1214147.058824
38    ANA    1998  velarra01   850000      1214147.058824
39    ANA    1998  wilsotr01   225000      1214147.058824
40    ANA    1998  watsoal01  2900000      1214147.058824
41    ANA    1998  walbema01   700000      1214147.058824
42    ANA    1998  delucri01   400000      1214147.058824
43    ANA    1998   hillke01  5000000      1214147.058824
44    ANA    1998  encaran01   180000      1214147.058824
45    ANA    1998  eenhoro01   182500      1214147.058824
46    ANA    1